In [2]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

/Users/divyam/Dev/Data_Science/DSF_Project/AcademicPaperRanking
['AcademicPaperRanking', '.DS_Store', 'dblp.csv', 'sys_argv.py', 'common_progress_kernel.ipynb', 'acm.csv', 'a', '2018_line_delimited.json', 'dblp.txt', 'aminer_papers_0.zip', 'acm.txt', '2018_data.json', 'aminer_papers_0']


In [2]:
#load dataframe from csv
df = pd.read_csv("../input/processed-acm-data-1/ranked_processed_acm_1.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824936 entries, 0 to 2824935
Data columns (total 11 columns):
Unnamed: 0           int64
PaperTitle           object
Authors              int64
Year                 int64
Publication_Venue    object
Index_Id             int64
References_Id        object
Abstract             object
count_reference      int64
page_rank            float64
Number Of Authors    int64
dtypes: float64(1), int64(6), object(4)
memory usage: 237.1+ MB


In [4]:
df.head()

,Unnamed: 0,PaperTitle,Authors,Year,Publication_Venue,Index_Id,References_Id,Abstract,count_reference,page_rank,Number Of Authors
0,117,Algorithms for trie compaction,910263,1984,ACM Transactions on Database Systems (TODS),118,:1157:252607:287497:289080:408503:326292:32687...,The trie data structure has many properties wh...,12,0.004683,2
1,117,Algorithms for trie compaction,147205,1984,ACM Transactions on Database Systems (TODS),118,:1157:252607:287497:289080:408503:326292:32687...,The trie data structure has many properties wh...,12,0.004683,2
2,133,"Logical, internal, and physical reference beha...",1086765,1984,ACM Transactions on Database Systems (TODS),134,:286819:287482:287680:287739:287766:287646:317...,This work investigates one aspect of the perfo...,9,0.005583,2
3,133,"Logical, internal, and physical reference beha...",394940,1984,ACM Transactions on Database Systems (TODS),134,:286819:287482:287680:287739:287766:287646:317...,This work investigates one aspect of the perfo...,9,0.005583,2
4,134,A parallel pipelined relational query processor,1087078,1984,ACM Transactions on Database Systems (TODS),135,:287004:287293:287666:287494:317954:321467:322...,This paper presents the design of a relational...,8,0.005590,3


In [11]:
paper_author = {}
for index,row in df.iterrows():
    count = paper_author.get(row["Index_Id"],0)
    count += 1
    paper_author[row["Index_Id"]] = count
print(max(paper_author.values()))

115


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_vals = list(df["Publication_Venue"].values.astype(str))
le.fit(train_vals)
df["Publication_Venue"] = le.transform(train_vals)

In [24]:
sorted(df["Year"].unique())

[1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

In [17]:
df["YearSincePublished"]=0
#YearSincePublished
current_year=2018
def yearsince(row):
    return current_year-row["Year"]
df["YearSincePublished"] = df.apply(lambda row:yearsince(row),axis = 1)

In [18]:
df.head()

,Unnamed: 0,PaperTitle,Authors,Year,Publication_Venue,Index_Id,References_Id,Abstract,count_reference,page_rank,Number Of Authors,YearSincePublished
0,117,Algorithms for trie compaction,910263,1984,637,118,:1157:252607:287497:289080:408503:326292:32687...,The trie data structure has many properties wh...,12,0.004683,2,34
1,117,Algorithms for trie compaction,147205,1984,637,118,:1157:252607:287497:289080:408503:326292:32687...,The trie data structure has many properties wh...,12,0.004683,2,34
2,133,"Logical, internal, and physical reference beha...",1086765,1984,637,134,:286819:287482:287680:287739:287766:287646:317...,This work investigates one aspect of the perfo...,9,0.005583,2,34
3,133,"Logical, internal, and physical reference beha...",394940,1984,637,134,:286819:287482:287680:287739:287766:287646:317...,This work investigates one aspect of the perfo...,9,0.005583,2,34
4,134,A parallel pipelined relational query processor,1087078,1984,637,135,:287004:287293:287666:287494:317954:321467:322...,This paper presents the design of a relational...,8,0.005590,3,34


In [27]:
num_columns= ["Authors","Year","Publication_Venue","Index_Id","count_reference","Number Of Authors","YearSincePublished","page_rank"]
new_df = df[num_columns]

In [32]:
df_train = new_df[new_df["Year"] <=2013]
train_x = df_train.drop(["page_rank"],axis=1)
train_y = df_train["page_rank"]
df_test =  new_df[new_df["Year"] > 2013]
test_x = df_test.drop(["page_rank"],axis=1)
test_y = df_test["page_rank"]

In [34]:
train_x.head()

,Authors,Year,Publication_Venue,Index_Id,count_reference,Number Of Authors,YearSincePublished
0,910263,1984,637,118,12,2,34
1,147205,1984,637,118,12,2,34
2,1086765,1984,637,134,9,2,34
3,394940,1984,637,134,9,2,34
4,1087078,1984,637,135,8,3,34


In [33]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(train_x,train_y)
print(reg.coef_)

[-8.52792208e-11 -2.19225534e-05 -1.54613851e-09 -4.55177327e-10
  1.92724447e-04  1.01802077e-05  2.19225534e-05]


In [35]:
from sklearn import metrics
pred_y = reg.predict(test_x)
rmse_initial = np.sqrt(metrics.mean_squared_error(pred_y,test_y))
print(rmse_initial)

0.0013098951832620973
